In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("/content/Crop.csv")

In [ ]:
df.drop(columns=['ph'],inplace=True)
df["X"]=np.random.randint(1,1000,size=len(df))
df["Y"]=np.random.randint(1,1000,size=len(df))


In [ ]:
df.fillna(df.median,inplace=True)
null_values=df.isnull().sum()
print(null_values)

N    0
P    0
K    0
X    0
Y    0
dtype: int64


In [ ]:
print(df.dtypes)

N    int64
P    int64
K    int64
X    int64
Y    int64
dtype: object


In [ ]:
"""for col in ['phosphorus', 'potassium', 'calcium', 'magnesium']:
    df[col] = pd.to_numeric(df[col], errors='coerce')"""

In [ ]:
df["height"]=np.random.randint(0,41,size=len(df))
df.head()

,N,P,K,X,Y,height
0,90,42,43,332,775,28
1,85,58,41,535,113,38
2,60,55,44,738,150,26
3,74,35,40,282,935,2
4,78,42,42,589,914,27


In [ ]:
df.rename(columns={'N':'0','P':'1','K':'2'},inplace=True)
df.head()

,0,1,2,X,Y,height
0,90,42,43,332,775,28
1,85,58,41,535,113,38
2,60,55,44,738,150,26
3,74,35,40,282,935,2
4,78,42,42,589,914,27


In [ ]:
normal_values = {
    '0': 2000 , #N
    '1': 1000, #P
    '2': 5000#K

}


In [ ]:
deficiency_scores = pd.DataFrame()
for mineral, min_val in normal_values.items():
    deficiency_scores[mineral] = np.maximum(0, min_val - df[mineral]) / min_val
    # 2️⃣ Identifier le minéral le plus carencé + score de carence total
df['Deficient_Mineral'] = deficiency_scores.idxmax(axis=1)
df['Deficiency_Score'] = deficiency_scores.max(axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
features = ['height', 'Deficiency_Score','Deficient_Mineral',"X","Y"]
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[features])

In [ ]:


from sklearn.cluster import AgglomerativeClustering
import plotly.express as px

import sqlite3

In [ ]:
clustering = AgglomerativeClustering(n_clusters=3, linkage='ward')
df['Cluster'] = clustering.fit_predict(scaled_data)

In [ ]:
fig = px.scatter(
    df, x='X', y='Y', color='Cluster', size='Deficiency_Score',
    hover_data=['Deficient_Mineral', 'height'],
    title='Landfill Clusters and Drill Points (2D, meters)'
)

fig.show()

In [ ]:
"""
# Constantes
SOIL_MASS_PER_SQM = 260  # kg/m²
ORANGE_N = 0.015         # 1.5%
ORANGE_P = 0.0015        # 0.15%
ORANGE_K = 0.025         # 2.5%

def calculate_orange_peels_cluster_logic(Deficiency_Score, Deficient_Mineral, area_hectares):

    delta_n = delta_p = delta_k = 0

    if  Deficient_Mineral == "0":
        delta_n = Deficiency_Score
    elif Deficient_Mineral== "1":
        delta_p = Deficiency_Score
    elif Deficient_Mineral == "2":
        delta_k = Deficiency_Score
    else:
        raise ValueError("Le nutriment manquant doit être 'N', 'P' ou 'K'.")


    mass_n = (delta_n / 1e6) * SOIL_MASS_PER_SQM / ORANGE_N
    mass_p = (delta_p / 1e6) * SOIL_MASS_PER_SQM / ORANGE_P
    mass_k = (delta_k / 1e6) * SOIL_MASS_PER_SQM / ORANGE_K


    required_mass_per_sqm = max(mass_n, mass_p, mass_k)


    area_sqm = area_hectares * 10_000

    total_mass = required_mass_per_sqm * area_sqm

    return required_mass_per_sqm, total_mass
""" # distribution

KeyboardInterrupt: Interrupted by user

In [ ]:
"""df["required_mass_per_sqm"] = df.apply(lambda row: calculate_orange_peels_cluster_logic(
    row["Deficiency_Score"], row["Deficient_Mineral"], row["Area_ha"]), axis=1)

# Calculer la moyenne par cluster
cluster_means = df.groupby("Cluster")["required_mass_per_sqm"].mean()

# Visualisation
plt.figure(figsize=(10, 6))
cluster_means.plot(kind='bar', color='orange')
plt.title("Quantité moyenne d'écorces d'orange requise par cluster (kg/m²)")
plt.xlabel("Cluster")
plt.ylabel("kg/m²")
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()
""" # visualisation